# Rasch (Nx1PL) MLTM
(Based on Cen 2009)
This model is used to explain the power law in learning.  In this notebook we try to build a neuralised version of the AFM and train it using simulated data.  The aim of using the AFM is to disentangle the latent traits that make up the overall score going into the sigmoid probability estimator.

The model is compensatory, which is a weakness.

In [11]:
from collections import defaultdict, Counter
from copy import copy
from math import exp, sqrt, log
from random import random, shuffle, choice, randint, uniform
import numpy

from keras import Input, Model
from keras.callbacks import EarlyStopping
from keras.constraints import non_neg, max_norm
from numpy import array, mean, ones
from pandas import concat
from pandas import DataFrame
from keras.models import Sequential
from keras.layers import LSTM, multiply, subtract, add, Activation, Lambda, Flatten
from keras.layers import Dense, concatenate, MaxPooling1D, LocallyConnected1D, Reshape
from keras.optimizers import Adam, SGD
from keras import backend as K
from keras import constraints

import tensorflow as tf

from utils import generate_student_name
import random

from matplotlib import pyplot as plt

n_traits = 2



In [12]:
generate_student_name()

'PHUDOY JECU '

In [13]:
from keras import backend as K
from keras.constraints import Constraint
from keras.engine.topology import Layer
from keras import initializers

class WeightClip(Constraint):
    '''Clips the weights incident to each hidden unit to be inside a range
    '''
    def __init__(self, min_w=0, max_w=4):
        self.min_w = min_w
        self.max_w = max_w

    def __call__(self, p):
        return K.clip(p, self.min_w, self.max_w)

    def get_config(self):
        return {'name': self.__class__.__name__,
                'min_w': self.min_w,
                'max_w': self.max_w }


class ProductLayer(Layer):

    def __init__(self, output_dim, kernel_constraint=WeightClip(min_w=-4.0, max_w=4.0), minv=-4,maxv=4, **kwargs):
        
        self.output_dim = output_dim
        super(ProductLayer, self).__init__(**kwargs)
        self.kernel_constraint= constraints.get(kernel_constraint)
        self.min_v = minv
        self.max_v = maxv

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(1, self.output_dim),
                                      initializer=initializers.RandomUniform(minval=self.min_v,maxval=self.max_v),
#                                       initializer=initializers.Constant(value=2.0),
                                      trainable=True,
                                      constraint=self.kernel_constraint)
        
        super(ProductLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        p = x * self.kernel
        print("shape p", p.shape)
        return p

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)
    
class DifferenceLayer(Layer):

    def __init__(self, output_dim, kernel_constraint=WeightClip(min_w=-4.0, max_w=4.0), minv=-4,maxv=4, invert=False, **kwargs):
        
        self.output_dim = output_dim
        super(DifferenceLayer, self).__init__(**kwargs)
        self.kernel_constraint= constraints.get(kernel_constraint)
        self.min_v = minv
        self.max_v = maxv
        self.invert = invert

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        initialiser = initializers.RandomUniform(minval=self.min_v,maxval=self.max_v)
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(1, self.output_dim),
                                      initializer=initialiser,
#                                       initializer=initializers.Constant(value=2.0),
                                      trainable=True,
                                      constraint=self.kernel_constraint)
        super(DifferenceLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        if self.invert:
            x = tf.Print(x, [x], message="x is:", first_n=-1, summarize=1024)
            k = tf.Print(self.kernel, [self.kernel], message="- kernel is:", first_n=-1, summarize=1024)
            p = x - k
        else:
            k = tf.Print(self.kernel, [self.kernel], message="kernel is:", first_n=-1, summarize=1024)
            x = tf.Print(x, [x], message="- x is:", first_n=-1, summarize=1024)
            p = k - x
#         p = K.print_tensor(p, message="p is:")
        p =  tf.Print(p, [p], message="p is:", first_n=-1, summarize=1024)
        print("shape p", p.shape)
        return p

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

In [14]:
for z in [-20, -10, -4,-3,-2,-1,0,1,2,3,4]:
    print(z, 1/(1+exp(-z)) )
    
    
# q_p_avg = 0.45

q_p_easiest = 0.95
q_p_hardest = 0.05

# pr_k_avg = q_p_avg**(1/n_traits)
# print("pr k avg:", pr_k_avg)

pr_k_easiest = q_p_easiest**(1/n_traits)
pr_k_hardest = q_p_hardest**(1/n_traits)

inv_sigmoid = lambda pr : ( -log((1/pr) -1) )
k_delta_easiest = inv_sigmoid(pr_k_easiest)
k_delta_hardest = inv_sigmoid(pr_k_hardest)

print("delz", k_delta_easiest, k_delta_hardest)
import math

spread = k_delta_easiest - k_delta_hardest
print("spread",spread)

beta_min = 0 #math.floor(0)
beta_max = 6 #math.ceil(beta_min + spread/2.0)

theta_min = 4 # math.floor(beta_max + spread/2.0)
theta_max = 11 # theta_min + math.ceil(k_delta_easiest)

print("th", theta_min, theta_max)
print("b", beta_min, beta_max)

max_comp_prob = 1/(1+exp(-(theta_max - beta_min)))
min_comp_prob = 1/(1+exp(-(theta_min - beta_max)))

print("max prob (rnd)", max_comp_prob)
print("min prob (rnd)", min_comp_prob)

print("max actl prob", (max_comp_prob)**n_traits)
print("min actl prob", (min_comp_prob)**n_traits)

-20 2.0611536181902037e-09
-10 4.5397868702434395e-05
-4 0.01798620996209156
-3 0.04742587317756678
-2 0.11920292202211755
-1 0.2689414213699951
0 0.5
1 0.7310585786300049
2 0.8807970779778823
3 0.9525741268224334
4 0.9820137900379085
delz 3.650491699884092 -1.2447699535774288
spread 4.895261653461521
th 4 11
b 0 6
max prob (rnd) 0.999983298578152
min prob (rnd) 0.11920292202211755
max actl prob 0.9999665974352416
min actl prob 0.014209336618611037


In [15]:
class Question():
    def __init__(self, qix, min_diff, max_diff, nt=None, nnw=None, optimiser=None):
        #self.MAX_BETA = 15
        self.id = qix
#         no_dummies = randint(1,(nt-1))
#         print("no_dummies=",no_dummies)
#         not_present=-20
#         min_diff = 0
#         max_diff = 10
#         self.betas = [ not_present for _ in range(nt) ]
        self.betas = [randint(min_diff,max_diff) for _ in range(nt)]
#         print("nt=",nt)
#         choices = [ x for x in range(nt) ]
#         for _ in range(no_dummies):
#             ch = random.choice(choices)
#             choices.remove(ch)
#             self.betas[ch]=randint(min_diff,max_diff)
        #print("Made q with betas:", self.betas)
        
        th = Input(shape=(nnw,), name="user_theta")
        b_in = Input(shape=(nnw,), name="beta_in")
        av_diff = (min_diff + max_diff)/2.0
#         bvar = ProductLayer(nnw
#                             , kernel_constraint=WeightClip(min_w=min_diff, max_w=max_diff), minv=0.9*av_diff,maxv=1.1*av_diff) # introduce nt trainable weights for the beta values
#         b = bvar(b_in)
        
        bvar = DifferenceLayer(nnw, kernel_constraint=WeightClip(min_w=min_diff, max_w=max_diff), minv=0.9*av_diff,maxv=1.1*av_diff, invert=True) # introduce nt trainable weights for the beta values)
        dif =bvar(th)
#         dif = subtract([th,b])
        print("dif",dif.shape)
#         Pr = Dense(1, activation="sigmoid")(dif)
        Prs = Lambda(lambda z: 1.0 / (1.0 + K.exp(-z)), name="qPr_sigmoid")(dif)
        print("Prs",Prs.shape)
        Pr = Lambda(lambda ps: K.prod(ps, axis=1, keepdims=True), name="qPr_prod")(Prs)
        print("Pr_prod",Pr.shape)
#         Pr = Reshape((1,))(Pr)
#         print("Pr",Pr.shape)
        model = Model(inputs=[th,b_in], outputs=Pr)
#         o = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
#         optimiser = Adam(lr=0.1)
        model.compile(optimizer=optimiser, loss="binary_crossentropy", metrics=["mse","accuracy"])
        self.model = model
        self.pred_betas = bvar
        print("q weights at init", model.get_weights())
        
        
# qs = [Question(qix, n_traits) for qix in range(10)]
# qs[0].model.summary()

# random.seed(666)
# qs = [Question(qix, n_traits) for qix in range(10)]
# for q in qs:
#     print(q.id, q.betas)

In [16]:
class Student():
    def __init__(self, min_abil, max_abil, nt=None, nnw=None, optimiser=None):
        #self.MAX_BETA = 15
        self.name = generate_student_name()
#         min_abil = 0
#         max_abil = 10
        self.thetas = [ randint(min_abil, max_abil) for _ in range(nt) ]
#         self.mastery = [0 for _ in range(nq)]
#         self.o_practice = [0 for _ in range(nq)]
#         self.h_practice = [0 for _ in range(nt)]
        #print("Made q with betas:", self.betas)
        th_in = Input(shape=(nnw,), name="theta_in")
        b = Input(shape=(nnw,), name="qn_beta")
        av_abil = (min_abil + max_abil)/2.0
#         th_var = ProductLayer(nnw, kernel_constraint=WeightClip(min_w=min_abil, max_w=max_abil), minv=0.9*av_abil, maxv=1.1*av_abil) # introduce nt trainable weights for the beta values

        th_var = DifferenceLayer(nnw, kernel_constraint=WeightClip(min_w=min_abil, max_w=max_abil), minv=0.9*av_abil,maxv=1.1*av_abil, invert=False) # introduce nt trainable weights for the beta values)
        dif = th_var(b)

#         th = th_var(th_in)
#         dif = subtract([th,b])
        print("dif",dif.shape)
        Prs = Lambda(lambda z: 1.0 / (1.0 + K.exp(-z)), name="sPr_sigmoid")(dif)
        print("Prs",Prs.shape)
        Pr = Lambda(lambda ps: K.prod(ps, axis=1, keepdims=True), name="sPr_prod")(Prs)
        print("Pr",Pr.shape)
#         Pr = Reshape((1,))(Pr)
        model = Model(inputs=[th_in,b], outputs=Pr)
#         o = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
#         optimiser= Adam(lr=0.1)
        model.compile(optimizer=optimiser, loss="binary_crossentropy", metrics=["mse","accuracy"])
#         print("s weights at init", model.get_weights())
        self.model = model
        self.pred_theta = th_var

In [17]:
def attempt_q(student: Student, q: Question):
    p = calculate_pass_probability(student.thetas, q.betas)
    this_att = uniform(0,1)
    if (this_att <= p):
        passed=True
#         student.mastery[q.id] = 1
    else:
        passed=False

    return passed

In [18]:
def calculate_pass_probability(thetas, betas):
    # additive factors model is:
    # p_pass = 1 / 1 + exp(-z)
    # where z = a + sum[1:n]( -b + gT )
    
    p_pass = 1.0
    print("th,b",thetas,betas)
    for th,b in zip(thetas,betas):
#         if b < 0:
#             continue
        z = (th-b)
        p_pass_step = 1.0 / (1.0 + exp(-z))
        p_pass *= p_pass_step # simple conjunctive model of success!
    try:        
        print("p_pass={}".format(p_pass))
    except OverflowError:
        p_pass = 0.0
    #print("real p_pass = {}".format(p_pass))
    return p_pass
    

In [19]:
qopt = Adam(lr=0.01)

def create_qs(n_qs, nt=n_traits, nnw=n_traits, optimiser=qopt):
    random.seed(666)
    master_qs = [Question(qix, beta_min,beta_max, nt=nt, nnw=nnw, optimiser=optimiser) for qix in range(n_qs)]
    for q in master_qs:
        nocomps = len(q.betas)
        mag = sqrt(sum([ pow(b, 2) for b in q.betas if b!=-10 ]))
        print("Q:{}, difficulty={:.2f} across {} components".format(q.id, mag, nocomps))
    return master_qs

# Training
This is where sh!t gets real.  We take our tr_len (1000?) students, and iterate over them 100 times to create 100,000 *complete examples* of a student attacking the curriculum.  The questions themselves are attacked in random order: the student has no intelligent guidance throught the material. (Obvious we may wish to provide that guidance at some point in the future.)

Remember, there are only 12 exercises in the curriculum, so if the student is taking 60 or 70 goes to answer them all, that's pretty poor.  But some of these kids are dumb as lumber, so cut them some slack!  They will all get there in the end since by the CMU AFM practice will, eventually, make perfect!

In [20]:
psi_opt = Adam()
def create_students(n_students, nt=n_traits, nnw=n_traits, optimiser=None):
    random.seed(666)
    psi_list = [ Student(theta_min,theta_max, nt=nt, nnw=nnw, optimiser=optimiser) for _ in range(n_students)]
    for psi in psi_list:
        print(psi.name, psi.thetas)
    return psi_list
    

In [21]:
extend_pop=False
extend_by = 90
if extend_pop:
    for _ in range(extend_by):
        nu_psi = Student(nt=n_traits, nq=len(master_qs), optimiser=psi_opt)
        psi_list.append(nu_psi)

In [22]:
import gc
def generate_attempts(master_qs, psi_list):
    attempts_by_psi = {}
    attempts_by_q = {}

    user_budget = 100
    user_patience = 10
    for run in range(1):
        print("----{}\n".format(run))
        for psi in psi_list:
            spend=0
            #psi.mastery = [0 for _ in range(nq)]
            qs = [ix for ix in range(len(master_qs))]
            print("* * * **** USER {}".format(psi.name))
            print("* * * * ** THETAS {}".format(psi.thetas))

            while(True):
                q_ct = 0
                qix = random.choice(qs)
                q = master_qs[qix]
                passed=False

                if psi.name not in attempts_by_psi:
                    attempts_by_psi[psi.name]=[]

                if q not in attempts_by_q:
                    attempts_by_q[q]=[]

                while not passed and q_ct<user_patience:
                    passed = attempt_q(psi, q)
                    tup = (psi, q, passed)
                    #attempts.append(tup)
                    attempts_by_psi[psi.name].append(tup)
                    attempts_by_q[q].append(tup)
                    q_ct+=1

                if passed:
                    print("passed")
                    qs.remove(qix)

                spend += 1

                if qs == [] or spend>=user_budget:
                        print("* ** *QFIN USER {}".format(psi.name))
                        break
    gc.collect()
    return attempts_by_psi, attempts_by_q

In [23]:
import os
import IPython

def calibrate(master_qs, psi_list, attempts_by_q, attempts_by_psi, n_iter=20, record_param_fit=False):
    es = EarlyStopping(monitor="loss", mode="auto")
    random.seed(666)
    min_mse = 1000
    min_avg_fit_rmse = 1000
    mses=[]
    accs=[]
    q_outer_mses = []
    q_outer_accs = []
    s_outer_mses = []
    s_outer_accs = []
    th_mses = []
    b_mses = []
    th_accs= []
    b_accs =[]
    avg_fit_rmses = []
    th_fit_rmses = []
    b_fit_rmses = []

    losses = []
    for ix in range(n_iter):
        gc.collect()
        q_clf_accs = []
        q_clf_mses = []
        q_clf_losses = []
#         for q in master_qs:
        q = random.choice(master_qs)
        if True:
            print(q.id)
            thetas = [ tup[0].pred_theta.get_weights()[0][0] for tup in attempts_by_q[q]]
            pfs = [ [tup[2]] for tup in attempts_by_q[q]]
            preb = q.pred_betas.get_weights()[0][0]
            
            arr_thetas = array(thetas)
            arr_pfs = array(pfs)
            arr_ones = numpy.ones( (len(thetas),len(thetas[0])) )            
            print("q arr_ones, shape:", arr_ones.shape)
            
            h = q.model.fit(x=[arr_thetas, arr_ones], y=arr_pfs, verbose=0, batch_size=1, callbacks=[es])
            del h
            #gc.collect()
            
            loss, mse, acc = q.model.evaluate(x=[arr_thetas, arr_ones], y=arr_pfs, verbose=0)
            q_clf_accs.append(acc)
            q_clf_mses.append(mse)
            q_clf_losses.append(loss)
            postb = q.pred_betas.get_weights()[0][0]
            print("q{} true({}), preb {} postb {}".format(q.id, q.betas, preb, postb))

        s_clf_accs=[]
        s_clf_mses=[]
        s_clf_losses = []
#         for _student in psi_list:
        _student = random.choice(psi_list)
        if True:
            betas = [ tup[1].pred_betas.get_weights()[0][0] for tup in attempts_by_psi[_student.name]]# if tup[1]==q]
            pfs = [ [tup[2]] for tup in attempts_by_psi[_student.name]]# if tup[1]==q]
            preth= _student.pred_theta.get_weights()[0][0] 

            arr_betas = array(betas)
            arr_pfs = array(pfs)
            arr_ones = numpy.ones( (len(betas),len(betas[0])) )
            print("student shape arr ones = ", arr_ones.shape)
            
            h = _student.model.fit(x=[arr_ones, arr_betas], y=arr_pfs, verbose=0, batch_size=1, callbacks=[es])
            del h
                        
            loss, mse, acc = _student.model.evaluate(x=[arr_ones, arr_betas], y=arr_pfs, verbose=0)
            s_clf_accs.append(acc)
            s_clf_mses.append(mse)
            s_clf_losses.append(loss)
            postth= _student.pred_theta.get_weights()[0][0]
            print("s{}: true({}), {} -> {}".format(_student.name, _student.thetas, preth, postth))

#         th_rmse = sqrt(mean([(numpy.sort(s.thetas) - numpy.sort(s.pred_theta.get_weights()[0][0]))**2 for s in psi_list]))
#         b_rmse = sqrt(mean([(numpy.sort(q.betas) - numpy.sort(q.pred_betas.get_weights()[0][0]))**2 for q in master_qs]))

#         avg_fit_rmse = (th_rmse + b_rmse)/2.0
        mse = (mean(s_clf_mses) + mean(q_clf_mses))/2.0
        acc = (mean(s_clf_accs) + mean(q_clf_accs)) /2.0
        loss = (mean(s_clf_losses) + mean(q_clf_losses)) /2.0
        
        if(record_param_fit):
            th_rmse = sqrt(mean([(numpy.sort(s.thetas) - numpy.sort(s.pred_theta.get_weights()[0][0]))**2 for s in psi_list]))
            b_rmse = sqrt(mean([(numpy.sort(q.betas) - numpy.sort(q.pred_betas.get_weights()[0][0]))**2 for q in master_qs]))

        print("iteration",ix)
#         print("b rmse:", b_rmse, "del:", (beta_rmses[-1] - b_rmse) if beta_rmses else 0)
#         print("th rmse: {} del: {}".format(th_rmse, (theta_rmses[-1] - th_rmse) if theta_rmses else 0))
        print("avg clf mse: {} del: {}".format(mse, (mses[-1] - mse) if mses else 0))
        print("avg clf loss {} del {}".format(loss, (losses[-1]-loss) if losses else 0))
        print("acc = {}".format(acc))
#         theta_rmses.append(th_rmse)
#         beta_rmses.append(b_rmse)
        mses.append(mse)
        accs.append(acc)

        if record_param_fit:
            th_fit_rmses.append(th_rmse)
            b_fit_rmses.append(b_rmse)
        
        th_mses.append(mean(s_clf_mses))
        b_mses.append(mean(q_clf_mses))
        th_accs.append(mean(s_clf_accs))
        b_accs.append(mean(q_clf_accs))
#         avg_fit_rmses.append(avg_fit_rmse)
#         s_outer_mses.append(mean(s_clf_mses))
#         s_outer_accs.append(mean(s_clf_accs))
#         q_outer_mses.append(mean(q_clf_mses))
#         q_outer_accs.append(mean(q_clf_accs))
        losses.append(loss)

#         if(min_fit_rmse > avg_fit_rmse):
#             min_fit_rmse = avg_fit_rmse
        
#         if(min_loss > loss):
#             min_loss = loss
#             patience_ct = 0
#         else:
#             patience_ct += 1
#         if patience_ct > max_pat:
#             print("patience exhausted:",max_pat,"iterations with no new clf MSE minimum")
#             break

        duration = .3  # second
        freq = 440  # Hz
#         os.system('play --no-show-progress --null --channels 1 synth %s sine %f' % (duration, freq))
        
#     print("final beta rmse = {}".format(beta_rmses[-1]))
#     print("final theta rmse = {}".format(theta_rmses[-1]))
#     print("final avg fit rmse = {}".format(avg_fit_rmses[-1]))
    print("final classifier mse = {}".format(mses[-1]))
    print("final acc = {}".format(accs[-1]))
    print("final loss = {}".format(losses[-1]))
    print("min mse =", min_mse)

    for m in [1, 1.5, 2, 2.5, 3]:
        duration = .1  # second
        freq = 440*m  # Hz
        os.system('play --no-show-progress --null --channels 1 synth %s sine %f' % (duration, freq))
#     jingle = IPython.display.Audio("calibration_complete.mp3", autoplay=True)
#     del jingle

    if record_param_fit:
        return th_fit_rmses,b_fit_rmses
    else:
        return (mses,accs, th_mses,th_accs, b_mses,b_accs)

In [25]:
nn_dimensions = [n_traits]
serieses = []
min_errs = []
n_qs = 10
n_students = 100
for ix,nnw in enumerate(nn_dimensions):
    qs = create_qs(n_qs, n_traits, nnw, optimiser=qopt)
    ss = create_students(n_students, n_traits, nnw, optimiser=qopt)
    attempts_by_psi, attempts_by_q = generate_attempts(qs,ss)
    print(qs[0].pred_betas.get_weights()[0][0])
    print(ss[0].pred_theta.get_weights()[0][0])
    th_rmses,b_rmses = calibrate(qs,ss,attempts_by_q, attempts_by_psi, n_iter=1000, record_param_fit=True)
#     serieses += resultz
    serieses.append( (th_rmses,b_rmses) )
#     if ix < len(serieses): #append to old series
#         (_mses,_accs) = serieses[ix]
#         _mses += mses
#         _accs += accs
#         serieuses[ix] = (_mses,_accs)
#     else: #create new series
#         serieses.append((mses,accs))
    gc.collect()
from pygame import mixer
mixer.init()
mixer.music.load('calibration_complete.mp3')
mixer.music.play()

xs = range(len(th_rmses))
# plt.plot(xs, numpy.multiply(1,theta_rmses), 'b--')
# plt.plot(xs, numpy.multiply(1,beta_rmses), 'b')
for th_rmses, b_rmses in serieses:
    plt.plot(xs, numpy.multiply(1,th_rmses), label="w=5, students"),
    plt.plot(xs, numpy.multiply(1,b_rmses), label="w=5, qns")

# plt.plot(xs, accs, "m")
# plt.plot(xs, numpy.multiply(1,s_outer_accs), "g")
# plt.plot(xs, numpy.multiply(1,q_outer_accs), "k")

# plt.plot(xs, numpy.multiply(1,mses), "m--")
# plt.plot(xs, numpy.multiply(1,s_outer_mses), "g--")
# plt.plot(xs, numpy.multiply(1,q_outer_mses), "k--")
fig = plt.gcf()
fig.set_size_inches(8, 5)
plt.xlabel("#iterations")
plt.ylabel("fit error (RMSE)")
plt.suptitle("Neural-MLTM Parameter Fitting")
plt.title("(skills=5, items=10, students=100)")
plt.legend()
plt.show()


shape p (?, 2)
dif (?, 2)
Prs (?, 2)
Pr_prod (?, 1)


KeyboardInterrupt: 

In [ ]:
n_traits = 5
nn_dimensions = [1,3,5,7,9]
serieses = []
min_errs = []
n_qs = 10
n_students = 100
for ix,nnw in enumerate(nn_dimensions):
    qs = create_qs(n_qs, n_traits, nnw, optimiser=qopt)
    ss = create_students(n_students, n_traits, nnw, optimiser=qopt)
    attempts_by_psi, attempts_by_q = generate_attempts(qs,ss)
    print(qs[0].pred_betas.get_weights()[0][0])
    print(ss[0].pred_theta.get_weights()[0][0])
    resultz = calibrate(qs,ss,attempts_by_q, attempts_by_psi, n_iter=70)
    serieses.append( resultz )
#     if ix < len(serieses): #append to old series
#         (_mses,_accs) = serieses[ix]
#         _mses += mses
#         _accs += accs
#         serieuses[ix] = (_mses,_accs)
#     else: #create new series
#         serieses.append((mses,accs))
    
from pygame import mixer
mixer.init()
mixer.music.load('calibration_complete.mp3')
mixer.music.play()

In [ ]:
# import pickle
# pickle.dump( serieses, open( "serieses.p", "wb" ) )

xmax=10
xs = range(len(serieses[0][0]))[0:xmax]
print(len(serieses))
for s in serieses:
    print(len(s))
# plt.plot(xs, numpy.multiply(1,theta_rmses), 'b--')
# plt.plot(xs, numpy.multiply(1,beta_rmses), 'b')
fig, axes = plt.subplots(nrows=3, ncols=2)
print(axes.shape)
for (mses,accs, th_mses,th_accs, b_mses,b_accs),c,d in zip(serieses,["r--","y--","k-","c--","b--"],nn_dimensions):
    axes[0,0].plot(xs, numpy.multiply(1,mses[0:xmax]), c, label="nnw={}".format(d))
    axes[0,1].plot(xs, numpy.multiply(1,accs[0:xmax]), c, label="nnw={}".format(d))
    axes[1,0].plot(xs, numpy.multiply(1,th_mses[0:xmax]), c, label="nnw={}".format(d))
    axes[1,1].plot(xs, numpy.multiply(1,th_accs[0:xmax]), c, label="nnw={}".format(d))
    axes[2,0].plot(xs, numpy.multiply(1,b_mses[0:xmax]), c, label="nnw={}".format(d))
    axes[2,1].plot(xs, numpy.multiply(1,b_accs[0:xmax]), c, label="nnw={}".format(d))

# plt.plot(xs, accs, "m")
# plt.plot(xs, numpy.multiply(1,s_outer_accs), "g")7
# plt.plot(xs, numpy.multiply(1,q_outer_accs), "k")

# plt.plot(xs, numpy.multiply(1,mses), "m--")
# plt.plot(xs, numpy.multiply(1,s_outer_mses), "g--")
# plt.plot(xs, numpy.multiply(1,q_outer_mses), "k--")
for ix in range(axes.shape[0]):
    subcats = ["Combined","Student","Question"]
    for iy in range(axes.shape[1]):
        axes[ix,iy].set_xlabel("#iterations")
        axes[ix,iy].legend()
        if iy==0:
            axes[ix,iy].set_title("{} fit error".format(subcats[ix]))
            axes[ix,iy].set_ylabel("fit error (RMSE)")
        else:
            axes[ix,iy].set_title("{} fit accuracy".format(subcats[ix]))
            axes[ix,iy].set_ylabel("prediction accuracy")

fig.suptitle("Neural-MLTM Parameter Fitting (k={}, q={}, s={})".format(n_traits, n_qs, n_students))
fig.set_size_inches(18, 18)

fig.show()

In [ ]:
xs = range(len(serieses[0][0]))
print(len(serieses))
for s in serieses:
    print(len(s))
# plt.plot(xs, numpy.multiply(1,theta_rmses), 'b--')
# plt.plot(xs, numpy.multiply(1,beta_rmses), 'b')
fig, axes = plt.subplots(nrows=1, ncols=2)
print(axes.shape)

min_mses = []
max_accs = []
bmin_mses = []
bmax_accs = []

for (mses,accs, th_mses,th_accs, b_mses,b_accs),c,d in zip(serieses,["r--","y--","k-","c--","b--"],nn_dimensions):
    mm = min(th_mses)
    ma = max(th_accs)
    min_mses.append(mm)
    max_accs.append(ma)
    mm = min(b_mses)
    ma = max(b_accs)
    bmin_mses.append(mm)
    bmax_accs.append(ma)
    

axes[0].plot(nn_dimensions, numpy.multiply(1,min_mses))
axes[0].plot(nn_dimensions, numpy.multiply(1,bmin_mses))
axes[1].plot(nn_dimensions, numpy.multiply(1,max_accs))
axes[1].plot(nn_dimensions, numpy.multiply(1,bmax_accs))
axes[0].axvline(x=5, linestyle="--")
axes[1].axvline(x=5, linestyle="--")

fig.suptitle("Neural-MLTM Parameter Fitting (k={}, q={}, s={})".format(n_traits, n_qs, n_students))
fig.set_size_inches(18,4)

fig.show()